<a href="https://colab.research.google.com/github/RehamHatem/NLP_Apply-CNN-architecture-on-Sentiment140-dataset/blob/main/NLP_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import nltk

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:


# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to preprocess text
def preprocess_text(text):
    # Remove Pre and Post Spaces
    text = str(text).strip()

    # Lower case the entire text
    text = str(text).lower()

    # Substitute New Line Characters with spaces
    text = re.sub(r"\n", r" ", text)

    # Tokenize the sentence
    word_tokens = word_tokenize(text)

    # Remove the punctuation and special characters from each individual word
    cleaned_text = []
    for word in word_tokens:
        cleaned_text.append("".join([char for char in word if char.isalnum()]))

    # Specify the stop words list
    stop_words = stopwords.words('english')

    # Remove the stopwords and words containing less than 2 characters
    text_tokens = [word for word in cleaned_text if (len(word) > 2) and (word not in stop_words)]

    # Lemmatize each word in the word list
    text = [lemmatizer.lemmatize(word) for word in text_tokens]

    return text

# Function to read CSV file and return the specified columns
def read_csv_file(filename, columns, headers, encoding='utf-8'):
    data = []
    data.append(headers)  # Append custom headers
    with open(filename, 'r', encoding=encoding) as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
            # Preprocess text column
            preprocessed_text = preprocess_text(row[5])  # Assuming the text is in the sixth column (0-based index)
            selected_row = [row[i] for i in columns]
            selected_row[1] = preprocessed_text  # Replace original text with preprocessed text
            data.append(selected_row)
    return data

# Define the column indices to extract
columns_to_extract = [0, 5]  # Assuming "Target" is the first column and "text" is the sixth column
custom_headers = ["target", "text"]  # Custom headers without "ids"

# Reading the CSV file
data = read_csv_file('/content/training.1600000.processed.noemoticon.csv', columns_to_extract, custom_headers, encoding='latin-1')

# Extracting features (text) and labels (target)
features = [row[1] for row in data[1:]]  # Exclude headers
labels = [int(row[0]) for row in data[1:]]  # Exclude headers and convert to integers

# Splitting the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Print the sizes of the training and testing sets
print("Training set size:", len(x_train))
print("Testing set size:", len(x_test))


Training set size: 1279999
Testing set size: 320000


In [ ]:
# Printing samples from training data
print("Training Data Samples:")
for i in range(5):
    print("Sample:", i+1)
    print("Text:", x_train[i])
    print("Label:", y_train[i])
    print()

# Printing samples from testing data
print("Testing Data Samples:")
for i in range(5):
    print("Sample:", i+1)
    print("Text:", x_test[i])
    print("Label:", y_test[i])
    print()

from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
encoder = LabelEncoder()

# Fit the encoder on the combined labels and transform both training and testing labels
y_combined = y_train + y_test
encoder.fit(y_combined)

y_train_encoded = encoder.transform(y_train)
y_test_encoded = encoder.transform(y_test)

# Convert integer labels to one-hot encoded format
num_classes = 3
y_train_onehot = to_categorical(y_train_encoded, num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes)

# # Print the shape of encoded labels
print("Shape of encoded training labels:", y_train_onehot.shape)
print("Shape of encoded testing labels:", y_test_onehot.shape)


Training Data Samples:
Sample: 1
Text: ['otavolimed', 'second', 'get', 'back', 'haha']
Label: 4

Sample: 2
Text: ['race', 'life', 'awesome']
Label: 4

Sample: 3
Text: ['good', 'clothes', 'otherwise', 'could', 'even', 'awkward', 'already', 'lol']
Label: 4

Sample: 4
Text: ['late', 'work', 'week', 'even', 'take', 'metro', 'guess', 'bus', 'affected', 'crash']
Label: 0

Sample: 5
Text: ['selestina118', 'incredible', 'people', 'need', 'know', 'worry']
Label: 4

Testing Data Samples:
Sample: 1
Text: ['nkluvr4eva', 'poor', 'little', 'dumpling', 'holmdel', 'vids', 'really', 'trying', 'hope', 'dont', 'try', 'hard', 'tonight']
Label: 0

Sample: 2
Text: ['bed', 'got', 'wake', 'hella', 'early', 'tomorrow', 'morning']
Label: 0

Sample: 3
Text: ['havent', 'able', 'listen', 'yet', 'speaker', 'busted']
Label: 0

Sample: 4
Text: ['remembers', 'solving', 'relatively', 'big', 'equation', 'two', 'unknown', 'total', 'pain', 'butt']
Label: 0

Sample: 5
Text: ['ate', 'much', 'feel', 'sick']
Label: 0

Shape o

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Assuming x_train, x_test, y_train, y_test are already defined

# Tokenize text data
max_words = 100000  # Assuming you want to consider only the top 10,000 words
max_len = 100  # Maximum sequence length

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)

x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Pad sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=max_len)
x_test_pad = pad_sequences(x_test_seq, maxlen=max_len)




In [ ]:
print(":", y_train_onehot[:5])

: [[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [ ]:

# Define the model
model = Sequential([
    Embedding(max_words, 100, input_length=max_len),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Softmax activation for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train_pad, y_train_onehot, epochs=2, batch_size=132,validation_data=(x_test_pad, y_test_onehot))



Epoch 1/2
9697/9697 [==============================] - 107s 11ms/step - loss: 0.4704 - accuracy: 0.7757 - val_loss: 0.4497 - val_accuracy: 0.7883
Epoch 2/2
9697/9697 [==============================] - 91s 9ms/step - loss: 0.4050 - accuracy: 0.8159 - val_loss: 0.4542 - val_accuracy: 0.7875


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test_pad, y_test_onehot)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


10000/10000 [==============================] - 30s 3ms/step - loss: 0.4536 - accuracy: 0.7853
Test Loss: 0.45364126563072205
Test Accuracy: 0.7852656245231628


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Define the model
filter_sizes = [3, 5, 7]  # Define the filter sizes you want to try

for filter_size in filter_sizes:
    model = Sequential([
        Embedding(max_words,100, input_length=max_len),
        Conv1D(128, filter_size, activation='relu'),  # Vary the filter size here
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(x_train_pad, y_train_onehot, epochs=3, batch_size=132, validation_data=(x_test_pad, y_test_onehot))

    # Evaluate the model
    loss, accuracy = model.evaluate(x_test_pad, y_test_onehot)
    print(f"Test Loss (Filter Size {filter_size}):", loss)
    print(f"Test Accuracy (Filter Size {filter_size}):", accuracy)


Epoch 1/3
9697/9697 [==============================] - 101s 10ms/step - loss: 0.4705 - accuracy: 0.7755 - val_loss: 0.4462 - val_accuracy: 0.7897
Epoch 2/3
9697/9697 [==============================] - 79s 8ms/step - loss: 0.4040 - accuracy: 0.8165 - val_loss: 0.4529 - val_accuracy: 0.7887
Epoch 3/3
10000/10000 [==============================] - 28s 3ms/step - loss: 0.4765 - accuracy: 0.7802
Test Loss (Filter Size 3): 0.47646859288215637
Test Accuracy (Filter Size 3): 0.7801968455314636
Epoch 1/3
9697/9697 [==============================] - 103s 10ms/step - loss: 0.4708 - accuracy: 0.7758 - val_loss: 0.4545 - val_accuracy: 0.7857
Epoch 2/3
9697/9697 [==============================] - 87s 9ms/step - loss: 0.4058 - accuracy: 0.8156 - val_loss: 0.4541 - val_accuracy: 0.7883
Epoch 3/3
10000/10000 [==============================] - 29s 3ms/step - loss: 0.4876 - accuracy: 0.7786
Test Loss (Filter Size 5): 0.48760154843330383
Test Accuracy (Filter Size 5): 0.7786437273025513
Epoch 1/3
9697/969

In [ ]:
# Take input text from the user
input_text = input("Enter a text to predict its label: ")

# Preprocess the input text
input_text = preprocess_text(input_text)
input_sequence = tokenizer.texts_to_sequences([input_text])
input_sequence_pad = pad_sequences(input_sequence, maxlen=max_len)

# Predict the label using the trained model
predicted_probabilities = model.predict(input_sequence_pad)
predicted_label = np.argmax(predicted_probabilities)

# Print the predicted label
print("Predicted Label:", predicted_label)


Enter a text to predict its label: I loved the movie! It was so engaging and entertaining
1/1 [==============================] - 0s 69ms/step
Predicted Label: 1


In [ ]:
# Define the number of sentences to test
num_sentences = 10
# Loop through each sentence index
for i in range(num_sentences):
    # Take input text from your data
    input_text = x_test[i]  # Assuming x_test contains the input text samples

    # Get the actual target label from your data
    actual_label = y_test[i]  # Assuming y_test contains the actual target labels

    # Preprocess the input text
    input_text = preprocess_text(input_text)
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_sequence_pad = pad_sequences(input_sequence, maxlen=max_len)

    # Predict the label using the trained model
    predicted_probabilities = model.predict(input_sequence_pad)
    predicted_label = np.argmax(predicted_probabilities)

    # Print the predicted and actual labels for each sentence
    print(f"Sentence {i+1}:")
    print("Input Text:", x_test[i])
    print("Predicted Label:", predicted_label)
    print("Actual Label:", actual_label)
    print()


1/1 [==============================] - 0s 17ms/step
Sentence 1:
Input Text: ['nkluvr4eva', 'poor', 'little', 'dumpling', 'holmdel', 'vids', 'really', 'trying', 'hope', 'dont', 'try', 'hard', 'tonight']
Predicted Label: 0
Actual Label: 0

1/1 [==============================] - 0s 16ms/step
Sentence 2:
Input Text: ['bed', 'got', 'wake', 'hella', 'early', 'tomorrow', 'morning']
Predicted Label: 0
Actual Label: 0

1/1 [==============================] - 0s 17ms/step
Sentence 3:
Input Text: ['havent', 'able', 'listen', 'yet', 'speaker', 'busted']
Predicted Label: 0
Actual Label: 0

1/1 [==============================] - 0s 18ms/step
Sentence 4:
Input Text: ['remembers', 'solving', 'relatively', 'big', 'equation', 'two', 'unknown', 'total', 'pain', 'butt']
Predicted Label: 0
Actual Label: 0

1/1 [==============================] - 0s 17ms/step
Sentence 5:
Input Text: ['ate', 'much', 'feel', 'sick']
Predicted Label: 0
Actual Label: 0

1/1 [==============================] - 0s 25ms/step
Sentence